<h1>Language Model</h1>

In [13]:
import timeit

import os
import nltk
import re
import pandas as pd
import string
import ast
from itertools import islice
import datetime

from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

def join_tags(tokens):
    for i in enumerate(tokens):
        hit = 0
        index = i[0]
        y = i[1]
        width = len(y)
        
        for x in y:
            if x is '-':
#                 save = tokens[index][hit+1:]
#                 tokens[index] = tokens[index][0:hit] + ''
#                 tokens[index+1] = save
#                 print(save)
#                 print(tokens[index])
                tokens[index] = tokens[index][0:hit] + ' ' + tokens[index][hit+1:]
            hit += 1
        
        if tokens[index] is ' ':
            tokens[index] = ''
        
        if tokens[index] == '  ':
            tokens[index] = ''
        
        if y[(width-1):width] is '.':
            tokens[index] = y[0:(width-1)]
        
        if tokens[index][0:1] is '-' or tokens[index][1:2] is '-':
            tokens[index] = ''
        
        if tokens[index][-2:-1] is "'":
            tokens[index] = tokens[index][0:-2]
        
        if  tokens[index][:1] is "'" :
            tokens[index] = tokens[index][1:]
        
        if tokens[index] is "s" or tokens[index] is "re":
            tokens[index-1] = tokens[index-1]
            tokens[index] = ''
        
        if tokens[index] is '<' and tokens[index+2] is '>':
            tokens[index+2] = '<'+ tokens[index+1]+'>'
            tokens[index] =''
            tokens[index+1] = ''
            
        if tokens[index] is '&' and tokens[index+2] is ';':
            tokens[index+2] = ''
            tokens[index] = ''
            tokens[index+1] = ''
            
        if tokens[index] is '(' or tokens[index] is ')':
            tokens[index] = ''
            
        if tokens[index][0:1] is ':':
            tokens[index] = ''
        
        if tokens[index][0:1] is '.' and tokens[index][1:2] is '.':
            tokens[index] = ''

        if y[(width-2):(width-1)] == '.' and y[(width-1):(width)] == '0':
            tokens[index] = y[0:(width-2)]
        
    tokens_clean = [x for x in tokens if x != '' and x not in ".,!?'" and x not in '<>']
        
    return tokens_clean

def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

def guess_date(string):
    for fmt in ["%Y/%m/%d", "%d-%m-%Y", "%Y%m%d", "%d-%b-%y", "%d-%m-%y", "%d-%b-%Y", "%d %b %y", "%d %m %y", "%d %b %Y"]:
        try:
            return str(datetime.datetime.strptime(string, fmt).date())
        except ValueError:
            continue
    return string

def preprocess(text_process):
    text_process = text_process.lower()
    tokens = word_tokenize(text_process)
    tokens_clean = join_tags(tokens)
    tokens_clean = [guess_date(i) for i in tokens_clean]
    stop_words = set(stopwords.words('english'))
    filtered_words = [guess_date(w) for w in tokens_clean if not w in stop_words]
    ps = PorterStemmer()
    stemmed_words = [ps.stem(w) for w in filtered_words]
    lemmatizer = WordNetLemmatizer()
    lemmatized_words = [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in stemmed_words]
    stemmed_text = ' '.join(stemmed_words)
    lemmatized_text = ' '.join(lemmatized_words)
    clean_text = lemmatized_text
    clean_words = sorted(list([lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in stemmed_words]))
    return clean_text, clean_words

def build_index_language_model():
    dict_term_doc = dict()
    clean_words_list = list()
    raw_words_list = list()
    dict_term_doc_final = dict()
    data = dict()
    key = list()
    value = list()
    doc_freqs = dict()
    
    start = timeit.default_timer()
    for index, i in enumerate(os.listdir('Data')):
        try:
            filename = i
            f = open('Data/' + filename, 'r')
            text = f.read()
            f.close()

            clean_text, clean_words = preprocess(text)
            clean_words_list.extend(clean_words)

            doc_freq = 0
            for w in clean_words:
                doc_freq+=1
                doc_freqs[filename[4:7]] = doc_freq
                if w not in dict_term_doc:
                    dict_term_doc[w] = [1, {filename[4:7]: 1}]
                else:
                    if filename[4:7] in dict_term_doc[w][1]:
                        dict_term_doc[w][1][filename[4:7]]+=1
                    else:
                        dict_term_doc[w][1][filename[4:7]]=1
                    dict_term_doc[w][0]+=1
        except Exception as e:
            print(str(e))
    
    stop = timeit.default_timer()
    print (stop-start)
    return stop-start

def language_model(query, limit, lambd):
    clean_query_text, clean_query_words = preprocess(query)
    print(clean_query_words)
    print(clean_query_text)
    result_dict = list()

    df_doc_freq = pd.read_csv('Document Frequencies.csv')
    df = pd.read_csv('Frequencies.csv')

    global_freq = int(open("global_frequency.txt", "r").read())
#     print(global_freq)

    start = timeit.default_timer()
    for i in df_doc_freq.iterrows():
        local_freq = i[1]['Frequency']
        doc_number = i[1]['Document']
        doc_number = str(doc_number).zfill(3)
        result = 1
        for j in clean_query_words:
            entries = df.loc[df['Term'] == j]['Documents'].values[0]
            entries = entries.split(', ',1)
            entries[0] = entries[0].replace("[", "")
            entries[1] = entries[1].replace(']', '')
            entries[1] = ast.literal_eval(entries[1])
            if doc_number in entries[1].keys():
                prob = (float(entries[1][doc_number])/float(local_freq)*lambd) + (float(entries[0])/float(global_freq)*(1-lambd))
            else:
                prob = (float(entries[0])/float(global_freq)*(1-lambd))
            result *= prob
        result_dict.append((doc_number, result))
    result_sorted = sorted(result_dict, key=lambda x: x[1], reverse=True)
    print(result_sorted)
    stop = timeit.default_timer()
    return result_sorted[:limit], stop-start

In [15]:
asd = build_index_language_model()

asd

145.32435439999995


145.32435439999995

In [17]:
language_model('accessed 9-Mar-87', 500, 1/2)

['1987-03-09', 'access']
access 1987-03-09
[('387', 4.788025715046517e-05), ('449', 4.788025715046517e-05), ('032', 2.478825242639475e-06), ('055', 2.478825242639475e-06), ('375', 2.4596880516608156e-06), ('265', 1.941595110250042e-06), ('144', 1.766401034860307e-06), ('134', 1.7592027148330282e-06), ('384', 1.568170375647554e-06), ('321', 1.4109529562719219e-06), ('443', 1.3230464422124287e-06), ('348', 1.0900119544739339e-06), ('146', 1.0362514400933324e-06), ('327', 9.074015116050426e-07), ('450', 9.074015116050426e-07), ('442', 8.891249260038669e-07), ('390', 8.54809704058802e-07), ('143', 8.231858720702127e-07), ('326', 8.082861819986659e-07), ('141', 6.961721206406334e-07), ('324', 6.857242152811034e-07), ('381', 6.857242152811034e-07), ('203', 6.563093432688884e-07), ('145', 6.294899011401042e-07), ('206', 6.294899011401042e-07), ('328', 6.294899011401042e-07), ('388', 6.210683371866211e-07), ('149', 6.128873893460948e-07), ('263', 6.128873893460948e-07), ('110', 5.9720723931841

([('387', 4.788025715046517e-05),
  ('449', 4.788025715046517e-05),
  ('032', 2.478825242639475e-06),
  ('055', 2.478825242639475e-06),
  ('375', 2.4596880516608156e-06),
  ('265', 1.941595110250042e-06),
  ('144', 1.766401034860307e-06),
  ('134', 1.7592027148330282e-06),
  ('384', 1.568170375647554e-06),
  ('321', 1.4109529562719219e-06),
  ('443', 1.3230464422124287e-06),
  ('348', 1.0900119544739339e-06),
  ('146', 1.0362514400933324e-06),
  ('327', 9.074015116050426e-07),
  ('450', 9.074015116050426e-07),
  ('442', 8.891249260038669e-07),
  ('390', 8.54809704058802e-07),
  ('143', 8.231858720702127e-07),
  ('326', 8.082861819986659e-07),
  ('141', 6.961721206406334e-07),
  ('324', 6.857242152811034e-07),
  ('381', 6.857242152811034e-07),
  ('203', 6.563093432688884e-07),
  ('145', 6.294899011401042e-07),
  ('206', 6.294899011401042e-07),
  ('328', 6.294899011401042e-07),
  ('388', 6.210683371866211e-07),
  ('149', 6.128873893460948e-07),
  ('263', 6.128873893460948e-07),
  ('110',

In [19]:
docs_results = language_model('accessed 9-Mar-87', 500, 1/2)
leng_lanss = len(docs_results[0])

documents = docs_results[0]

numbers = []
score = []

for key,value in documents:
    numbers.append(key)
    score.append(value)

numbers

['1987-03-09', 'access']
access 1987-03-09
[('387', 4.788025715046517e-05), ('449', 4.788025715046517e-05), ('032', 2.478825242639475e-06), ('055', 2.478825242639475e-06), ('375', 2.4596880516608156e-06), ('265', 1.941595110250042e-06), ('144', 1.766401034860307e-06), ('134', 1.7592027148330282e-06), ('384', 1.568170375647554e-06), ('321', 1.4109529562719219e-06), ('443', 1.3230464422124287e-06), ('348', 1.0900119544739339e-06), ('146', 1.0362514400933324e-06), ('327', 9.074015116050426e-07), ('450', 9.074015116050426e-07), ('442', 8.891249260038669e-07), ('390', 8.54809704058802e-07), ('143', 8.231858720702127e-07), ('326', 8.082861819986659e-07), ('141', 6.961721206406334e-07), ('324', 6.857242152811034e-07), ('381', 6.857242152811034e-07), ('203', 6.563093432688884e-07), ('145', 6.294899011401042e-07), ('206', 6.294899011401042e-07), ('328', 6.294899011401042e-07), ('388', 6.210683371866211e-07), ('149', 6.128873893460948e-07), ('263', 6.128873893460948e-07), ('110', 5.9720723931841

['387',
 '449',
 '032',
 '055',
 '375',
 '265',
 '144',
 '134',
 '384',
 '321',
 '443',
 '348',
 '146',
 '327',
 '450',
 '442',
 '390',
 '143',
 '326',
 '141',
 '324',
 '381',
 '203',
 '145',
 '206',
 '328',
 '388',
 '149',
 '263',
 '110',
 '446',
 '205',
 '270',
 '385',
 '382',
 '444',
 '329',
 '116',
 '323',
 '148',
 '147',
 '330',
 '386',
 '389',
 '447',
 '325',
 '209',
 '142',
 '267',
 '204',
 '202',
 '210',
 '266',
 '264',
 '150',
 '322',
 '441',
 '269',
 '201',
 '383',
 '268',
 '445',
 '261',
 '262',
 '208',
 '207',
 '448',
 '001',
 '002',
 '003',
 '004',
 '005',
 '006',
 '007',
 '008',
 '009',
 '010',
 '011',
 '012',
 '013',
 '014',
 '015',
 '016',
 '017',
 '018',
 '019',
 '020',
 '021',
 '022',
 '023',
 '024',
 '025',
 '026',
 '027',
 '028',
 '029',
 '030',
 '031',
 '033',
 '034',
 '035',
 '036',
 '037',
 '038',
 '039',
 '040',
 '041',
 '042',
 '043',
 '044',
 '045',
 '046',
 '047',
 '048',
 '049',
 '050',
 '051',
 '052',
 '053',
 '054',
 '056',
 '057',
 '058',
 '059',
 '060',


In [21]:
import xml.dom.minidom

results = []
# read xml
for key,value in documents:
    dictionary = dict()
    filename = 'Doc0' + key + '.xml'
    
    print('filename: ', filename)
    
    doc = xml.dom.minidom.parse('C:/Users/Asus/Downloads/Git/search_engine/XML/' + filename)
    title = doc.getElementsByTagName('TITLE')
    title = title[0].firstChild.nodeValue

    date = doc.getElementsByTagName('DATE')
    date = date[0].firstChild.nodeValue
    body = doc.getElementsByTagName('BODY')
    body = body[0].firstChild.nodeValue
  
    dictionary['document_no'] = key
    dictionary['score'] = value
    dictionary['title'] = title
    dictionary['date'] = date
    dictionary['body'] = body
    results.append(dictionary)
    
len(results)

filename:  Doc0387.xml
filename:  Doc0449.xml
filename:  Doc0032.xml
filename:  Doc0055.xml
filename:  Doc0375.xml
filename:  Doc0265.xml
filename:  Doc0144.xml
filename:  Doc0134.xml
filename:  Doc0384.xml
filename:  Doc0321.xml
filename:  Doc0443.xml
filename:  Doc0348.xml
filename:  Doc0146.xml
filename:  Doc0327.xml
filename:  Doc0450.xml
filename:  Doc0442.xml
filename:  Doc0390.xml
filename:  Doc0143.xml
filename:  Doc0326.xml
filename:  Doc0141.xml
filename:  Doc0324.xml
filename:  Doc0381.xml
filename:  Doc0203.xml
filename:  Doc0145.xml
filename:  Doc0206.xml
filename:  Doc0328.xml
filename:  Doc0388.xml
filename:  Doc0149.xml
filename:  Doc0263.xml
filename:  Doc0110.xml
filename:  Doc0446.xml
filename:  Doc0205.xml
filename:  Doc0270.xml
filename:  Doc0385.xml
filename:  Doc0382.xml
filename:  Doc0444.xml
filename:  Doc0329.xml
filename:  Doc0116.xml
filename:  Doc0323.xml
filename:  Doc0148.xml
filename:  Doc0147.xml
filename:  Doc0330.xml
filename:  Doc0386.xml
filename:  

filename:  Doc0336.xml
filename:  Doc0337.xml
filename:  Doc0338.xml
filename:  Doc0339.xml
filename:  Doc0340.xml
filename:  Doc0341.xml
filename:  Doc0342.xml
filename:  Doc0343.xml
filename:  Doc0344.xml
filename:  Doc0345.xml
filename:  Doc0346.xml
filename:  Doc0347.xml
filename:  Doc0349.xml
filename:  Doc0350.xml
filename:  Doc0351.xml
filename:  Doc0352.xml
filename:  Doc0353.xml
filename:  Doc0354.xml
filename:  Doc0355.xml
filename:  Doc0356.xml
filename:  Doc0357.xml
filename:  Doc0358.xml
filename:  Doc0359.xml
filename:  Doc0360.xml
filename:  Doc0361.xml
filename:  Doc0362.xml
filename:  Doc0363.xml
filename:  Doc0364.xml
filename:  Doc0365.xml
filename:  Doc0366.xml
filename:  Doc0367.xml
filename:  Doc0368.xml
filename:  Doc0369.xml
filename:  Doc0370.xml
filename:  Doc0371.xml
filename:  Doc0372.xml
filename:  Doc0373.xml
filename:  Doc0374.xml
filename:  Doc0376.xml
filename:  Doc0377.xml
filename:  Doc0378.xml
filename:  Doc0379.xml
filename:  Doc0380.xml
filename:  

500

<h1>TF IDF</h1>

In [23]:
import os
import nltk
import re
import pandas as pd
import string
import ast
from itertools import islice
import math

def build_index_tfidf():
    df_tfidf = pd.read_csv('Konstruksi Indeks.csv')
    doc = df_tfidf['Documents']
    term = df_tfidf['Term']
    
    
    dict_tfidf = dict()
    dict_doc = dict()
    list_doc = list()
    dic = dict()
    max_range = len(term)
    N = 500
    
    starts = timeit.default_timer()
    for i in range (0,max_range):
        start = 2
        end = 5
        count = 1
        for y in range (0,int(len(doc[i])/7)):
            if(int(len(doc[i])/7)!=1):
                if(doc[i][start:end] != doc[i][(start-7):(end-7)]):
                    dict_doc[term[i]+" "+doc[i][start:end]] = count
                    dic[doc[i][start:end]] = count
                else:
                    dict_doc[term[i]+" "+doc[i][start:end]] = count
                    dic[doc[i][start:end]] = count
                    count+=1
            else :
                dict_doc[term[i]+" "+doc[i][start:end]] = count
                dic[doc[i][start:end]] = count
            start = start+7
            end = end+7
            dict_tfidf[term[i]] = count
            
    stop = timeit.default_timer()
#     print(stop-starts)

def tfidf(query):
    df_tfidf = pd.read_csv('Konstruksi Indeks.csv')
    doc = df_tfidf['Documents']
    term = df_tfidf['Term']
    
    text, data = preprocess(query)
    z=1
    count = 1
    N = 500+1
    max_range = len(term)

    dict_w = dict()
    dict_f = dict()
    dict_tf = dict()
    dict_idf = dict()
    dict_dj = dict()
    dict_sim = dict()

    for q in data:
        for i in range (0,max_range):
            ni = 0
            start = 2
            end = 5
            count = 1
            if (term[i] == q):
                for y in range (0,int(len(doc[i])/7)):
                    if(int(len(doc[i])/7)!=1):
                        if(doc[i][start:end] != doc[i][(start-7):(end-7)]):
                            ni+=1
                            dict_f['f',doc[i][start:end]]=count
                        else:
                            dict_f['f',doc[i][start:end]]=count
                            count+=1
                            ni=1
                    else:
                        dict_f['f',doc[i][start:end]]=count
                        ni+=1

                    start = start+7
                    end = end+7

                dict_idf[q] = math.log((1+N/ni),10)


    for key in dict_f:
        dict_tf['tf',z] = 1+math.log(dict_f[key],10)
        z+=1

    z=1

    for key in dict_tf:
        for key2 in dict_idf:
            dict_w['w',z] = dict_tf[key]*dict_idf[key2]
            z+=1

    z=1

    for key in dict_w:
        dict_dj['dj',z] = math.sqrt(pow(dict_w[key],2))
        z+=1

    z=1
    for key in dict_dj:
        for key2 in dict_w:
            dict_sim["d",z] = dict_w[key2]/dict_dj[key]
            z+=1
            break
    stop = timeit.default_timer()
    
    result = [(i[1], j)for i,j in dict_f.items()]
    
    print(stop-start)
    
    return result, stop-start 

In [25]:
results = tfidf('Chairman closed the last')

results[0]

1224.6432245


[('018', 1),
 ('028', 5),
 ('052', 1),
 ('061', 6),
 ('098', 8),
 ('102', 4),
 ('107', 8),
 ('155', 4),
 ('161', 4),
 ('195', 13),
 ('227', 16),
 ('231', 5),
 ('233', 6),
 ('234', 6),
 ('255', 19),
 ('261', 6),
 ('263', 7),
 ('322', 39),
 ('330', 7),
 ('355', 7),
 ('392', 7),
 ('399', 46),
 ('404', 8),
 ('409', 9),
 ('420', 9),
 ('430', 9),
 ('438', 51),
 ('441', 9),
 ('457', 9),
 ('488', 9),
 ('492', 59),
 ('494', 59),
 ('095', 1),
 ('111', 1),
 ('133', 9),
 ('135', 1),
 ('150', 11),
 ('153', 11),
 ('154', 11),
 ('156', 3),
 ('188', 4),
 ('194', 12),
 ('204', 5),
 ('208', 15),
 ('232', 6),
 ('254', 17),
 ('269', 33),
 ('276', 34),
 ('318', 7),
 ('324', 8),
 ('371', 8),
 ('382', 44),
 ('419', 10),
 ('439', 11),
 ('446', 52),
 ('495', 60),
 ('001', 1),
 ('012', 1),
 ('026', 3),
 ('040', 6),
 ('043', 6),
 ('047', 6),
 ('060', 6),
 ('069', 6),
 ('078', 7),
 ('114', 8),
 ('116', 8),
 ('134', 9),
 ('136', 10),
 ('138', 10),
 ('145', 10),
 ('148', 10),
 ('169', 11),
 ('174', 11),
 ('178', 11

In [26]:
valuess = []

rs = results[0]
dr = dict()

for key, value in results[0]:
    valuess.append(value)

asd = rs.sort()

valuess

[1,
 5,
 1,
 6,
 8,
 4,
 8,
 4,
 4,
 13,
 16,
 5,
 6,
 6,
 19,
 6,
 7,
 39,
 7,
 7,
 7,
 46,
 8,
 9,
 9,
 9,
 51,
 9,
 9,
 9,
 59,
 59,
 1,
 1,
 9,
 1,
 11,
 11,
 11,
 3,
 4,
 12,
 5,
 15,
 6,
 17,
 33,
 34,
 7,
 8,
 8,
 44,
 10,
 11,
 52,
 60,
 1,
 1,
 3,
 6,
 6,
 6,
 6,
 6,
 7,
 8,
 8,
 9,
 10,
 10,
 10,
 10,
 11,
 11,
 11,
 11,
 13,
 14,
 14,
 14,
 16,
 16,
 16,
 16,
 16,
 17,
 17,
 20,
 21,
 28,
 30,
 31,
 32,
 34,
 35,
 36,
 36,
 36,
 36,
 37,
 38,
 40,
 40,
 40,
 42,
 43,
 43,
 43,
 43,
 43,
 44,
 44,
 45,
 46,
 46,
 50,
 52,
 52,
 52,
 52,
 54,
 55,
 55,
 55,
 58,
 59,
 59,
 62,
 63]

In [27]:
def build_index_tf_idf():
    dict_term_doc = dict()
    clean_words_list = list()
    raw_words_list = list()
    dict_term_doc_final = dict()
    data = dict()
    key = list()
    value = list()
    
    start = timeit.default_timer()
    for i in os.listdir('Data'):
        try:
            filename = i
            f = open('Data/' + filename, 'r')
            text = f.read()
            f.close()

            clean_text, clean_words = preprocess(text)
            clean_words_list.extend(clean_words)
            for w in clean_words:
                if w not in dict_term_doc:
                    dict_term_doc[w] = [filename[4:7]]
                else:
                    dict_term_doc[w].append(filename[4:7])
    #         target = 'Clean/' + filename[:-3] +'xml'
    #         new_file = open(target, 'w+')
    #         new_file.write(clean_text)
    #         new_file.close()
        except Exception as e:
            print(str(e))

    for i in sorted(dict_term_doc.keys()):
        dict_term_doc_final[i] = dict_term_doc[i]

    df_term_doc = pd.DataFrame(dict_term_doc_final.items(), columns = ['Term', 'Documents'])

    df_term_doc.to_csv('Konstruksi Indeks.csv')
    stop = timeit.default_timer()
    
    print(stop-start)
    return stop-start

In [28]:
build_index_tf_idf()

146.89489460000004


146.89489460000004

In [29]:
def cobain(kueri):
    query = kueri
    user_input = query.lower()
    docs_results = tfidf(user_input)
    times = docs_results[1]
            
    documents = docs_results[0]
    founded = len(documents)
                
    results = []
    
    result_dict = list()
    
    for key,value in documents:
        result_dict.append((key, value))
        result_sorted = sorted(result_dict, key=lambda x: x[1], reverse=True)
    
    for key, value in result_sorted:
        dictionary = dict()
        filename = 'Doc0' + key + '.xml'
        print('filename: ', filename)
        # if display == 'original':
        doc = xml.dom.minidom.parse('C:/Users/Asus/Downloads/Git/search_engine/XML/' + filename)
        title = doc.getElementsByTagName('TITLE')
        title = title[0].firstChild.nodeValue

        date = doc.getElementsByTagName('DATE')
        date = date[0].firstChild.nodeValue
        body = doc.getElementsByTagName('BODY')
        body = body[0].firstChild.nodeValue

        dictionary['document_no'] = key
        dictionary['score'] = value
        dictionary['title'] = title
        dictionary['date'] = date
        dictionary['body'] = body
        results.append(dictionary) 
    
    print(results)
        
cobain('Chairman closed the last')

1372.1032396
filename:  Doc0498.xml
filename:  Doc0496.xml
filename:  Doc0495.xml
filename:  Doc0492.xml
filename:  Doc0494.xml
filename:  Doc0484.xml
filename:  Doc0491.xml
filename:  Doc0463.xml
filename:  Doc0454.xml
filename:  Doc0455.xml
filename:  Doc0458.xml
filename:  Doc0451.xml
filename:  Doc0446.xml
filename:  Doc0444.xml
filename:  Doc0445.xml
filename:  Doc0447.xml
filename:  Doc0448.xml
filename:  Doc0438.xml
filename:  Doc0425.xml
filename:  Doc0399.xml
filename:  Doc0386.xml
filename:  Doc0405.xml
filename:  Doc0383.xml
filename:  Doc0382.xml
filename:  Doc0376.xml
filename:  Doc0378.xml
filename:  Doc0343.xml
filename:  Doc0350.xml
filename:  Doc0367.xml
filename:  Doc0372.xml
filename:  Doc0375.xml
filename:  Doc0340.xml
filename:  Doc0328.xml
filename:  Doc0338.xml
filename:  Doc0339.xml
filename:  Doc0322.xml
filename:  Doc0316.xml
filename:  Doc0314.xml
filename:  Doc0288.xml
filename:  Doc0292.xml
filename:  Doc0303.xml
filename:  Doc0306.xml
filename:  Doc0284.xm